# 性能テスト結果分析

このノートブックは `performance_test.ipynb` で実行された性能テストの結果を分析し、可視化します。

## 分析内容
1. **データの読み込みと前処理**: `analysis/data` から最新のテスト結果を読み込みます。
2. **全体サマリー**: 全テスト結果の概要を表示します。
3. **サービス間性能比較**: レスポンス時間やスループット(QPS)をサービス間で比較します。
4. **負荷パターン別分析**: 負荷（軽・中・高）による性能の変化を分析します。
5. **スケーラビリティ分析**: 負荷増加に対する各サービスの性能維持能力を評価します。
6. **リソース使用量分析**: コンテナのCPU・メモリ使用量を分析します。
7. **レポート生成**: 分析結果をまとめたMarkdownレポートを生成し、`analysis/report` に保存します。

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import glob
from datetime import datetime

# Plotlyのデフォルトテーマを設定
import plotly.io as pio
pio.templates.default = "plotly_white"

# パス設定
BASE_DIR = Path("/Users/codefox/workspace/practice_infra_arch/key_value_store")
DATA_DIR = BASE_DIR / "analysis" / "data"
REPORT_DIR = BASE_DIR / "analysis" / "report"
REPORT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ 必要なライブラリがインポートされました")
print(f"データディレクトリ: {DATA_DIR}")
print(f"レポートディレクトリ: {REPORT_DIR}")

## 1. データの読み込みと前処理

In [ ]:
def load_latest_test_data(data_dir):
    """最新の包括的テスト結果(JSON)を読み込む"""
    list_of_files = glob.glob(str(data_dir / 'comprehensive_test_*.json'))
    if not list_of_files:
        print("❌ テスト結果ファイルが見つかりません。")
        return None
    
    latest_file = max(list_of_files, key=os.path.getctime)
    print(f"📂 最新のテスト結果ファイルを読み込み中: {os.path.basename(latest_file)}")
    
    with open(latest_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # エラーを含む結果を除外してDataFrameに変換
    valid_data = [d for d in data if 'error' not in d]
    df = pd.DataFrame(valid_data)
    
    if df.empty:
        print("⚠️ 有効なテストデータがありません。")
        return None
        
    # データ型の整理
    numeric_cols = [
        'clients', 'total_requests', 'successful_requests', 'duration', 
        'error_rate', 'qps', 'avg_response_time', 'median_response_time',
        'p95_response_time', 'p99_response_time', 'min_response_time', 'max_response_time'
    ]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
    # 負荷パターンの順序を定義
    df['pattern_name'] = pd.Categorical(df['pattern_name'], categories=['light', 'medium', 'heavy'], ordered=True)
    
    print(f"✅ {len(df)}件の有効なテスト結果を読み込みました。")
    return df

df_results = load_latest_test_data(DATA_DIR)

if df_results is not None:
    df_results.head()

## 2. 全体サマリー

In [ ]:
if df_results is not None:
    # サマリーテーブルの表示
    summary = df_results.pivot_table(
        index='service_name',
        columns='pattern_name',
        values=['qps', 'avg_response_time', 'error_rate'],
        aggfunc='mean'
    ).fillna(0)
    
    # カラムの整形
    summary.columns = [f'{val}_{cat}' for val, cat in summary.columns]
    
    # 表示形式の調整
    summary_styled = summary.style.background_gradient(cmap='viridis', low=0.1, high=0.9, axis=None).format({
        col: '{:.2f}' for col in summary.columns if 'qps' in col or 'avg_response_time' in col
    }).format({
        col: '{:.2f}%' for col in summary.columns if 'error_rate' in col
    }).set_caption("📊 全体パフォーマンスサマリー")
    
    display(summary_styled)
else:
    print("データがありません。")

## 3. サービス間性能比較

In [ ]:
if df_results is not None:
    fig = px.bar(
        df_results,
        x='service_name',
        y='qps',
        color='pattern_name',
        barmode='group',
        title='🚀 サービス別スループット(QPS)比較',
        labels={'service_name': 'サービス名', 'qps': 'QPS (Queries Per Second)', 'pattern_name': '負荷パターン'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_layout(xaxis_title="サービス名", yaxis_title="QPS (高いほど良い)")
    fig.show()
else:
    print("データがありません。")

In [ ]:
if df_results is not None:
    fig = px.bar(
        df_results,
        x='service_name',
        y='avg_response_time',
        color='pattern_name',
        barmode='group',
        title='⏱️ サービス別平均レスポンス時間比較',
        labels={'service_name': 'サービス名', 'avg_response_time': '平均レスポンス時間 (ms)', 'pattern_name': '負荷パターン'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_layout(xaxis_title="サービス名", yaxis_title="平均レスポンス時間 (ms, 低いほど良い)")
    fig.show()
else:
    print("データがありません。")

In [ ]:
if df_results is not None:
    df_melted = df_results.melt(
        id_vars=['service_name', 'pattern_name'],
        value_vars=['p95_response_time', 'p99_response_time'],
        var_name='percentile',
        value_name='response_time'
    )
    
    fig = px.bar(
        df_melted,
        x='service_name',
        y='response_time',
        color='percentile',
        facet_col='pattern_name',
        barmode='group',
        title='レスポンス時間パーセンタイル比較 (95%tile, 99%tile)',
        labels={'service_name': 'サービス名', 'response_time': 'レスポンス時間 (ms)', 'percentile': 'パーセンタイル'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_yaxes(matches=None)
    fig.show()
else:
    print("データがありません。")

## 4. スケーラビリティ分析

In [ ]:
if df_results is not None:
    fig = px.line(
        df_results.sort_values('pattern_name'),
        x='pattern_name',
        y='qps',
        color='service_name',
        markers=True,
        title='📈 負荷増加に伴うQPSの変化 (スケーラビリティ)',
        labels={'pattern_name': '負荷パターン', 'qps': 'QPS', 'service_name': 'サービス名'}
    )
    fig.update_layout(xaxis_title="負荷パターン", yaxis_title="QPS (高いほど良い)")
    fig.show()
else:
    print("データがありません。")

In [ ]:
if df_results is not None:
    fig = px.line(
        df_results.sort_values('pattern_name'),
        x='pattern_name',
        y='avg_response_time',
        color='service_name',
        markers=True,
        title='📉 負荷増加に伴う平均レスポンス時間の変化',
        labels={'pattern_name': '負荷パターン', 'avg_response_time': '平均レスポンス時間 (ms)', 'service_name': 'サービス名'}
    )
    fig.update_layout(xaxis_title="負荷パターン", yaxis_title="平均レスポンス時間 (ms, 低いほど良い)")
    fig.show()
else:
    print("データがありません。")

## 5. リソース使用量分析

注意: この分析は `performance_test.ipynb` でリソース情報が正しく収集されている場合にのみ有効です。

In [ ]:
def parse_resource_stats(df):
    """リソース統計情報をパースしてDataFrameに追加"""
    if 'final_stats' not in df.columns:
        print("⚠️ リソース統計情報が見つかりません。")
        return df
    
    cpu_usage = []
    mem_usage = []
    
    for index, row in df.iterrows():
        stats = row['final_stats']
        if isinstance(stats, dict) and stats:
            # 'app'コンテナの統計を優先
            app_container_key = next((k for k in stats.keys() if 'app' in k), None)
            if app_container_key:
                cpu_usage.append(stats[app_container_key].get('cpu_percent', 0))
                mem_usage.append(stats[app_container_key].get('memory_usage', 0) / (1024*1024)) # MBに変換
            else: # 'app'が見つからない場合は最初のコンテナ
                first_key = list(stats.keys())[0]
                cpu_usage.append(stats[first_key].get('cpu_percent', 0))
                mem_usage.append(stats[first_key].get('memory_usage', 0) / (1024*1024)) # MBに変換
        else:
            cpu_usage.append(np.nan)
            mem_usage.append(np.nan)
            
    df['cpu_percent'] = cpu_usage
    df['memory_usage_mb'] = mem_usage
    return df

if df_results is not None:
    df_resources = parse_resource_stats(df_results.copy())
    if 'cpu_percent' in df_resources.columns:
        display(df_resources[['service_name', 'pattern_name', 'cpu_percent', 'memory_usage_mb']].head())
else:
    print("データがありません。")

In [ ]:
if df_results is not None and 'cpu_percent' in df_resources.columns:
    fig = px.bar(
        df_resources.dropna(subset=['cpu_percent']),
        x='service_name',
        y='cpu_percent',
        color='pattern_name',
        barmode='group',
        title='💻 サービス別CPU使用率',
        labels={'service_name': 'サービス名', 'cpu_percent': 'CPU使用率 (%)', 'pattern_name': '負荷パターン'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_layout(xaxis_title="サービス名", yaxis_title="CPU使用率 (%)")
    fig.show()
else:
    print("CPUデータがありません。")

In [ ]:
if df_results is not None and 'memory_usage_mb' in df_resources.columns:
    fig = px.bar(
        df_resources.dropna(subset=['memory_usage_mb']),
        x='service_name',
        y='memory_usage_mb',
        color='pattern_name',
        barmode='group',
        title='🧠 サービス別メモリ使用量',
        labels={'service_name': 'サービス名', 'memory_usage_mb': 'メモリ使用量 (MB)', 'pattern_name': '負荷パターン'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_layout(xaxis_title="サービス名", yaxis_title="メモリ使用量 (MB)")
    fig.show()
else:
    print("メモリデータがありません。")

## 6. レポート生成

In [ ]:
def generate_report(df):
    """Markdown形式のレポートを生成"""
    if df is None:
        return "# 性能テストレポート\n\nテストデータが見つかりませんでした。"
    
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    report = f"# 性能テスト分析レポート\n\n生成日時: {timestamp}\n\n"
    
    report += "## 1. 総合評価\n\n"
    
    # QPSとレスポンス時間でランキング
    heavy_load_df = df[df['pattern_name'] == 'heavy'].sort_values('qps', ascending=False)
    if not heavy_load_df.empty:
        report += "### 高負荷時 QPSランキング\n"
        for i, row in heavy_load_df.iterrows():
            report += f"1. **{row['service_name']}**: {row['qps']:.2f} QPS\n"
        report += "\n"
        
        report += "### 高負荷時 平均レスポンス時間ランキング (短い順)\n"
        for i, row in heavy_load_df.sort_values('avg_response_time').iterrows():
            report += f"1. **{row['service_name']}**: {row['avg_response_time']:.2f} ms\n"
        report += "\n"
    
    report += "## 2. サマリーテーブル\n\n"
    summary_df = df.pivot_table(
        index='service_name',
        columns='pattern_name',
        values=['qps', 'avg_response_time']
    ).fillna(0)
    report += summary_df.to_markdown(floatfmt=".2f")
    report += "\n\n"
    
    report += "## 3. 設計パターン別特性分析\n\n"
    for service_name in df['service_name'].unique():
        report += f"### {service_name}\n"
        service_df = df[df['service_name'] == service_name]
        light = service_df[service_df['pattern_name'] == 'light'].iloc[0]
        heavy = service_df[service_df['pattern_name'] == 'heavy'].iloc[0]
        
        qps_growth = (heavy['qps'] / light['qps']) if light['qps'] > 0 else 0
        resp_growth = (heavy['avg_response_time'] / light['avg_response_time']) if light['avg_response_time'] > 0 else 0
        
        report += f"- **スループット**: 軽負荷時 {light['qps']:.2f} QPS → 高負荷時 {heavy['qps']:.2f} QPS ({qps_growth:.2f}倍)\n"
        report += f"- **レスポンス時間**: 軽負荷時 {light['avg_response_time']:.2f} ms → 高負荷時 {heavy['avg_response_time']:.2f} ms ({resp_growth:.2f}倍)\n"
        
        if resp_growth > 5:
            report += f"- **考察**: ⚠️ 高負荷時にレスポンス時間が大幅に悪化しています。ボトルネックの可能性があります。\n"
        elif qps_growth < 2 and heavy['clients']/light['clients'] > 5:
            report += f"- **考察**: ⚠️ 負荷を増やしてもスループットが伸び悩んでいます。スケーラビリティに課題があるかもしれません。\n"
        else:
            report += f"- **考察**: ✅ 負荷に対して比較的安定した性能を維持しています。\n"
        report += "\n"
        
    return report

if df_results is not None:
    report_content = generate_report(df_resources)
    
    # レポートをファイルに保存
    report_filename = f"performance_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    report_path = REPORT_DIR / report_filename
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(report_content)
        
    print(f"✅ レポートが生成されました: {report_path}")
    
    # レポートのプレビュー
    print("\n--- レポートプレビュー ---")
    print(report_content[:1000] + "...")
else:
    print("データがないためレポートを生成できません。")